In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from keras.utils import np_utils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/down_sample.csv')[['Subreddit', 'simp_lem']]

In [4]:
df.head()

,Subreddit,simp_lem
0,0,i hate how i feel when i get interested in som...
1,0,how to make friend can someone suggest somethi...
2,0,so today wa my birthday 9 11 and a i wa hoping...
3,0,having a hard time just need someone to say i ...
4,0,anti depressant inducing trip like state any o...


In [5]:
X = df.drop(columns='Subreddit')
y = df['Subreddit']

X_train, X_test, y_train, y_test=train_test_split(X, y, stratify = y, random_state=42)

In [6]:
cvec = CountVectorizer()
cvec.fit(X_train['simp_lem'])
X_train_cv = pd.DataFrame(cvec.transform(X_train['simp_lem']).toarray(), columns = cvec.get_feature_names())
X_test_cv = pd.DataFrame(cvec.transform(X_test['simp_lem']).toarray(), columns = cvec.get_feature_names())

tvec = TfidfVectorizer()
tvec.fit(X_train['simp_lem'])
X_train_tv = pd.DataFrame(tvec.transform(X_train['simp_lem']).todense(), columns = tvec.get_feature_names())
X_test_tv = pd.DataFrame(tvec.transform(X_test['simp_lem']).todense(), columns = tvec.get_feature_names())

In [7]:
sc = StandardScaler()
sc.fit(X_train_cv)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:
X_train_cv_sc = sc.transform(X_train_cv)

In [9]:
X_test_cv_sc = sc.transform(X_test_cv)

In [10]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_cv_sc.shape[1]))
model.add(Dropout(0.25))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision()], optimizer='adam')

In [11]:
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, 
                           verbose=1, mode='auto')

In [12]:
history = model.fit(X_train_cv_sc, y_train, 
                    validation_data=(X_test_cv_sc, y_test),
                    epochs=50, 
                    batch_size=64,
                    verbose=1, 
                    callbacks=[early_stop])

Epoch 1/50
352/352 [==============================] - 5s 12ms/step - loss: 0.8920 - accuracy: 0.6108 - precision: 0.6065 - val_loss: 0.5452 - val_accuracy: 0.7215 - val_precision: 0.7292
Epoch 2/50
352/352 [==============================] - 2s 7ms/step - loss: 0.5302 - accuracy: 0.7607 - precision: 0.7597 - val_loss: 0.5566 - val_accuracy: 0.7193 - val_precision: 0.7088
Epoch 3/50
352/352 [==============================] - 2s 7ms/step - loss: 0.3614 - accuracy: 0.8500 - precision: 0.8394 - val_loss: 0.6613 - val_accuracy: 0.7148 - val_precision: 0.6947
Epoch 4/50
352/352 [==============================] - 2s 7ms/step - loss: 0.2471 - accuracy: 0.9009 - precision: 0.8931 - val_loss: 0.8476 - val_accuracy: 0.7216 - val_precision: 0.7437
Epoch 5/50
352/352 [==============================] - 2s 7ms/step - loss: 0.1726 - accuracy: 0.9353 - precision: 0.9292 - val_loss: 1.0430 - val_accuracy: 0.7179 - val_precision: 0.7165
Epoch 6/50
352/352 [==============================] - 2s 7ms/step - l